In [1]:
import sys
import os

# upstage-ai-advanced-ir7 디렉토리의 경로를 생성합니다
project_dir = os.path.join(os.getcwd(), 'upstage-ai-advanced-ir7')

# 시스템 경로에 upstage-ai-advanced-ir7 디렉토리를 추가합니다
sys.path.append(project_dir)

In [2]:
from tools.searchengine import SearchEngine
from tools.query_transformer import QueryTransformer
from tools.generate import Generate

import json
import torch

import gc

from dotenv import load_dotenv
import os

/opt/conda/envs/myvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
def clean_vram():
    gc.collect()
    torch.cuda.empty_cache()

clean_vram()   

In [4]:
# 환경 변수 설정
load_dotenv('/upstage-ai-advanced-ir7/.env')
upstage_api_key = os.getenv('UPSTAGE_API_KEY')
os.environ["OPENAI_API_KEY"] = upstage_api_key

# 사용자 설정
embedding_dim = 4096
query_transformer_model_name = "rtzr/ko-gemma-2-9b-it"
generate_model_name = "rtzr/ko-gemma-2-9b-it"
faiss_path = '/upstage-ai-advanced-ir7/data/faiss_index.faiss'

# tool 생성
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
search_engine = SearchEngine(embedding_dim, upstage_api_key, device)
query_transformer = QueryTransformer(query_transformer_model_name, device)
generator = Generate(generate_model_name, device)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 10/10 [00:09<00:00,  1.01it/s]


# Index 생성 OR 로드

In [5]:
with open("/upstage-ai-advanced-ir7/data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

# index 만들기 or 가져오기
search_engine.create_elasticsearch_index(docs)
search_engine.load_faiss_index(faiss_path, docs)
clean_vram()


elastic search client 생성중 ...

elastic search client 생성 완료 ... 

faiss index load 중 ...

faiss index load 완료!


# Query 입력

In [6]:
prompt_input = input("무엇을 도와드릴까요?")
query = [{"role": 'user', "content": prompt_input}]

chat_prompt = query_transformer.create_chat_prompt(query)
standalone_query = query_transformer.generate_standalone_query(chat_prompt)
clean_vram()
print(standalone_query)

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
/opt/conda/envs/myvenv/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


자연어 처리 기술의 음성 인식 능력에 대한 정보


# 관련 문서 검색(RAG)

In [11]:
clean_vram()
eval_id = "123"
query_data = [{"eval_id": eval_id, 'standalone_query': standalone_query}]
result = search_engine.search_quries(query_data)[0]
clean_vram()

print(result)

  0%|          | 0/1 [00:00<?, ?it/s]

reranking 실행 시작 ...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
100%|██████████| 1/1 [00:07<00:00,  7.09s/it]

{'eval_id': '123', 'standalone_query': '자연어 처리 기술의 음성 인식 능력에 대한 정보', 'topk': ['d1900d8f-e489-4922-973e-1d7b9729bcb1', 'f72070a4-124c-49a7-9871-68d9fe3ee469', 'e1c86893-206b-43d2-8900-b1dd40aeefa5'], 'references': [{'score': 0.05344495549798012, 'content': '노이즈 캔슬링 기기는 소리의 간섭을 사용하여 작동합니다. 간섭은 두 개 이상의 소리 파동이 만나면 발생하는 현상으로, 서로 반대 방향으로 진동하는 파동이 만나면 상쇄되어 소리가 사라지는 원리입니다. 노이즈 캔슬링 기기는 주변의 잡음을 감지하고, 그와 동일한 크기와 반대 방향으로 소리를 생성하여 상쇄시킵니다. 이를 통해 주변의 잡음을 효과적으로 제거하고, 조용한 환경을 조성할 수 있습니다. 노이즈 캔슬링 기기는 주로 항공기, 자동차, 헤드폰 등에서 사용되며, 사용자에게 편안하고 조용한 환경을 제공합니다.'}, {'score': 0.049043845385313034, 'content': '20.0 T 자기장에서 31P의 NMR 주파수는 345.0 MHz입니다. 핵자기공명(NMR)은 분자의 화학적 환경과 구조에 대한 정보를 제공하는 분광 기술입니다. NMR에서 핵이 공명하는 주파수는 핵이 노출되는 자기장의 세기에 따라 다릅니다. 이 경우 31P 핵은 20.0 T 자기장에 노출되어 345.0 MHz의 공명 주파수가 발생합니다. 이 정보는 과학자들이 분자의 특성과 행동을 자세히 연구할 수 있도록 해주기 때문에 화학, 생화학, 의학 연구 등 다양한 분야에서 가치가 있습니다.'}, {'score': 0.03491341322660446, 'content': 'DFA(Deterministic Finite Automaton)는 문자열의 패턴을 인식하는 데 사용되는 수학적 모델입니다. 이 경우 우리는 DFA가 언어 L을 인식하는

# 생성

In [12]:
clean_vram()
query = result['standalone_query']
references = result['references']
response = generator.generate_response(query, references)
print(response)

 참조 문헌 2에 따르면 자연어 처리 기술의 음성 인식 능력에 대한 정보는 없습니다. 



